In [ ]:
import requests
import json


In [ ]:
requests.delete("http://localhost:8983/solr/tmdb/schema/model-store/test")

In [ ]:
requests.delete("http://localhost:8983/solr/tmdb/schema/feature-store/test")

In [ ]:
feature_set = [
    {
      "name" : "title_bm25",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "title:(${keywords})"
      }
    },
    {
      "name" : "overview_bm25",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    }
]

requests.put('http://localhost:8983/solr/tmdb/schema/feature-store', 
             json=feature_set)

In [ ]:
resp = requests.get('http://localhost:8983/solr/tmdb/schema/feature-store/test')
resp.json()

In [ ]:
params = {
    "fl": "id,title,[features store=test efi.keywords=\"rambo\"]",
    'q': "id:7555 OR id:1370 id:1369 OR id:1368",
    'rows': 10,
    'wt': 'json'  
}

resp = requests.post('http://localhost:8983/solr/tmdb/select', data=params).json()
resp

In [ ]:
model = """## LambdaMART
## No. of trees = 10
## No. of leaves = 10
## No. of threshold candidates = 256
## Learning rate = 0.1
## Stop early = 100

<ensemble>
	<tree id="1" weight="0.1">
		<split>
			<feature> 2 </feature>
			<threshold> 10.664251 </threshold>
			<split pos="left">
				<feature> 1 </feature>
				<threshold> 0.0 </threshold>
				<split pos="left">
					<output> -1.8305741548538208 </output>
				</split>
				<split pos="right">
					<feature> 2 </feature>
					<threshold> 9.502127 </threshold>
					<split pos="left">
						<feature> 1 </feature>
						<threshold> 7.0849166 </threshold>
						<split pos="left">
							<output> 0.23645669221878052 </output>
						</split>
						<split pos="right">
							<output> 1.7593677043914795 </output>
						</split>
					</split>
					<split pos="right">
						<output> 1.9719607830047607 </output>
					</split>
				</split>
			</split>
			<split pos="right">
				<feature> 2 </feature>
				<threshold> 0.0 </threshold>
				<split pos="left">
					<output> 1.3728954792022705 </output>
				</split>
				<split pos="right">
					<feature> 2 </feature>
					<threshold> 8.602512 </threshold>
					<split pos="left">
						<feature> 1 </feature>
						<threshold> 0.0 </threshold>
						<split pos="left">
							<feature> 2 </feature>
							<threshold> 13.815164 </threshold>
							<split pos="left">
								<output> 1.9401178359985352 </output>
							</split>
							<split pos="right">
								<output> 1.99532949924469 </output>
							</split>
						</split>
						<split pos="right">
							<feature> 1 </feature>
							<threshold> 11.085816 </threshold>
							<split pos="left">
								<output> 2.0 </output>
							</split>
							<split pos="right">
								<output> 1.99308180809021 </output>
							</split>
						</split>
					</split>
					<split pos="right">
						<output> 1.9870178699493408 </output>
					</split>
				</split>
			</split>
		</split>
	</tree>
</ensemble>
"""

from ltr.helpers.convert import convert

feature_mapping = [{'name': 'title_bm25'}, {'name': 'overview_bm25'}]

solr_model = convert(model, 'test', 'test', feature_mapping)
print(json.dumps(solr_model, indent=2))

In [ ]:
requests.put('http://localhost:8983/solr/tmdb/schema/model-store',
              json=solr_model)

In [ ]:
params = {
    'q': 'rambo',
    'rows': 10,
    "rq": "{!ltr model=test efi.keywords=\"rambo\"}",
    'wt': 'json',
    'fl': 'title'
}

resp = requests.post('http://localhost:8983/solr/tmdb/select', data=params).json()
resp